# Installing the **Dependencies**

In [3]:
!pip install langchain langchain-community langchain-core langchain-text-splitters
!pip install chromadb
!pip install sentence-transformers

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepa

# **Install and Start Ollama service in the background and pull Mistral 7B**

In [26]:
!curl -fsSL https://ollama.ai/install.sh|sh
!nohup ollama serve >/dev/null 2>&1 &
import time
time.sleep(5) # Give Ollama some time to start
!ollama pull mistral

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



# **Creating the Speech.txt File**

In [28]:
speech_text="""
The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to continue to be\nheld as sacred and infallible? You must take a stand against the scriptures. Either you must stop the practice of caste or you must stop believing\nin the shastras. You cannot have both. The problem of caste is not a problem of social reform. It is a problem of overthrowing the authority of the\nshastras. So long as people believe in the sanctity of the shastras, they will never be able to get rid of caste. The work of social reform is like the\nwork of a gardener who is constantly pruning the leaves and branches of a tree without ever attacking the roots. The real enemy is the belief in the\nshastras.
"""
with open("speech.txt","w") as f:
  f.write(speech_text)

# **Importing all the required Libraries**

In [18]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

# **Load and Split the Document intoo Chunks**

In [11]:
loader= TextLoader("speech.txt")
doc=loader.load()

langchain_text_splitters=CharacterTextSplitter(chunk_size=300,chunk_overlap=50)
chunks=langchain_text_splitters.split_documents(doc)
len(chunks)

for i, chunk in enumerate(chunks):
  print(f"--- Chunk {i+1} ---")
  print(chunk.page_content)
  print("\n"+"="*50+"\n")

--- Chunk 1 ---
The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to continue to be
held as sacred and infallible? You must take a stand against the scriptures. Either you must stop the practice of caste or you must stop believing
in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It is a problem of overthrowing the authority of the
shastras. So long as people believe in the sanctity of the shastras, they will never be able to get rid of caste. The work of social reform is like the
work of a gardener who is constantly pruning the leaves and branches of a tree without ever attacking the roots. The real enemy is the belief in the
shastras.




# **Build Embeddings and chromaDB Vector Store**

In [12]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore=Chroma.from_documents(chunks,embedding=embeddings,collection_name="ambedkar_rag")
retriever=vectorstore.as_retriever()


/tmp/ipython-input-381583462.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Initialising Mistral 7B via Ollama**

In [16]:
llm=Ollama(model="mistral")

# **Building the RAG Retrieval QA CHAIN**

In [19]:
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever)

# **ASK Your Questions**

In [20]:
summary_question = "What is the main argument of the speech and what is the real remedy suggested?"
result = qa.invoke(summary_question)
print(result["result"])

 The main argument of the speech is that the issue of caste in Indian society cannot be addressed through social reform alone, but requires a more fundamental change: challenging and overthrowing the authority of the shastras (scriptures). The real remedy suggested is to destroy the belief in the sanctity of the shastras as they are seen as the source of the caste system.


In [27]:
question = "what is the real enemy according to the speech?"
result = qa.invoke(question)
print(result["result"])

 The real enemy, according to the given speech, is the belief in the sanctity of the shastras.


In [29]:
question = "what does the gardener analogy means?"
result = qa.invoke(question)
print(result["result"])

 The gardener analogy means that social reform, which aims to address issues like caste, is similar to a gardener who continually prunes leaves and branches of a tree but never attacks its roots. In this context, the "tree" represents the deeply ingrained belief in the sanctity of the shastras (scriptures). The gardener's pruning (social reform) may provide temporary solutions or improvements, but as long as the root cause (the belief in the shastras) remains, the issue will persist. To truly solve the problem of caste, one must challenge and overthrow the authority of the shastras, just as a gardener would attack the roots of a tree to eliminate it completely.


In [30]:
question = "why must belief in shastras be destroyed?"
result = qa.invoke(question)
print(result["result"])

 Belief in shastras must be destroyed because, as per the given context, the problem of caste persists due to the belief in the sanctity and infallibility of the shastras. The author argues that so long as people believe in their authority, they will never be able to get rid of caste. Thus, to eradicate caste, the belief in the shastras needs to be overthrown.
